# Update IBI Scores Table

In [ ]:
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection
import ipywidgets as widgets
import pandas as pd
import io
import warnings

def update_table(b):    
    with output:
        # Clear any previous output
        output.clear_output()
        
        # Initialize progress bar
        progress = widgets.IntProgress(value=0, min=0, max=4, step=1, description='Progress:')
        display(progress)
        
        # Check if file is uploaded
        if uploader.value:
            
            try:
                # Read in uploaded file as a csv
                uploaded_file = next(iter(uploader.value.values()))
                csv = io.BytesIO(uploaded_file['content'])

                # Read uploaded file into a Pandas DataFrame
                df = pd.read_csv(csv)
                
                # Drop null values (+ last 5 Salesforce lines)
                df.dropna(subset=['Site Name'], inplace=True)
                
                # Most recent ibi scores for each site
                recent_ibi = df.copy()
                recent_ibi['Session_Date'] = pd.to_datetime(recent_ibi['Mon_Date_Time'])
                recent_ibi = recent_ibi[recent_ibi.groupby('WQMSite ID')['Session_Date'].transform('max') == recent_ibi['Session_Date']]
                recent_ibi = recent_ibi.rename(columns={'WQMSite ID': 'WQMSite_ID', 'Avg_IBI_Score': 'Index_of_Biological_Integrity__'})
                recent_ibi = recent_ibi[['WQMSite_ID', 'Index_of_Biological_Integrity__']]

                # Grab Sites layer from ArcGIS
                items = gis.content.search(query='title:Sites', item_type='Feature Service')
                for item in items:
                    if item.title == "Sites":
                        collection = FeatureLayerCollection.fromitem(item)
                        sites_df = collection.layers[0].query().df

                        # Merge the DataFrames on 'WQMSite_ID', keeping all rows from sites_df
                        sites_df = pd.merge(sites_df.drop(columns=['Index_of_Biological_Integrity__']), recent_ibi, how='left', on='WQMSite_ID')

                        # Update progress
                        progress.value = 1
                        
                        # Upload the csv to files
                        sites_df.to_csv('sites.csv', index=False)
                        csv_path = '/arcgis/sites.csv'
                        collection.manager.overwrite(csv_path)
                        os.remove('/arcgis/sites.csv')
                        
                        # Update progress
                        progress.value = 2

                # Rename IBI, add Score column        
                df = df.rename(columns={'Avg_IBI_Score': 'Index_of_Biological_Integrity__'})
                
                # Upload the csv to files
                df.to_csv('ibi_scores.csv', index=False)
                
                # Update progress
                progress.value = 3
                           
                # Search for IBI Scores table
                items = gis.content.search(query='title:IBI Scores', item_type='Feature Service')
                ibi_found = False
                for item in items:
                    if item.title == "IBI Scores":
                        ibi_found = True
                        
                        # Replace table item
                        collection = FeatureLayerCollection.fromitem(item)
                        csv_path = '/arcgis/ibi_scores.csv'
                        collection.manager.overwrite(csv_path)
                        
                # IBI Scores table not found, create new table
                if not ibi_found:
                    item_properties = {
                        "title": "IBI Scores",
                        'type': 'CSV',
                    }
                    n_item = gis.content.add(item_properties=item_properties, data='/arcgis/ibi_scores.csv')
                    p_item = n_item.publish()
                
                # Clean up temporary file
                os.remove('/arcgis/ibi_scores.csv')
            
                # Update progress
                progress.value = 4

                print('IBI Scores table updated.')
            
            except Exception as e:
                print(f"Error: {str(e)}")
                        
        else:
            print("Error: No file uploaded.")
            
# Suppress admin warning
warnings.filterwarnings("ignore", message="You are logged on as .* with an administrator role")
            
# GIS connection
gis = GIS("home")            
            
# Initialize widgets and button click event
output = widgets.Output()
uploader = widgets.FileUpload(accept='.csv', multiple=False)
button = widgets.Button(description='Update Table', tooltip='Click me')
button.on_click(update_table)

# Display widgets and button
print('\nUpload LimnoTech IBI Over Time report (csv format).')
display(uploader)

print('\nClick the button to update the table.')
display(button)
display(output)